In [2]:
import numpy as np

In [3]:
import random

In [4]:
def choose_in_range(a,L,H):
    if a<L:
        return L
    if a>H:
        return H
    if L<=a<=H:
        return a

In [5]:
def choose_another(i,m):
    j=i
    while i==j:
        j=np.random.randint(0,m)
    return j
        

In [6]:
def smo(train,train_label,C,toler,maxiter):
    m,n=train.shape
    alpha=np.zeros((m,1))
    w=np.random.randn(n,1)
    b=0
    Iter=0
    while Iter<maxiter:
        change_two=0
        for i in range(m):
            f_i=train[i].dot(w)+b
            E_i=f_i-train_label[i]
            if  (E_i*train_label[i]<-toler and alpha[i]<C) or (E_i*train_label[i]>toler and alpha[i]>0):
                j=choose_another(i,m)
                f_j=train[j].dot(w)+b
                E_j=f_j-train_label[j]
                alpha_i_old=alpha[i].copy()
                alpha_j_old=alpha[j].copy()
                if train_label[i]!=train_label[j]:
                    L=max(0,alpha[j]-alpha[i])
                    H=min(C,alpha[j]-alpha[i]+C)
                else:
                    L=max(0,alpha[i]+alpha[j]-C)
                    H=min(C,alpha[i]+alpha[j])
                if L==H:
                    continue
                eta=2*train[i].dot(train[j].T)-train[i].dot(train[i].T)-train[j].dot(train[j].T)
                if eta>=0:
                    continue
                alpha[j]=alpha[j]-train_label[j]*(E_i-E_j)/eta
                alpha[j]=choose_in_range(j,L,H)
                if abs(alpha[j]-alpha_j_old)<0.000001:
                    continue
                alpha[i]=alpha[i]-train_label[i]*train_label[j]*(alpha[j]-alpha_j_old)
                b1=b-E_i-train_label[i]*(alpha[i]-alpha_i_old)*train[i].dot(train[i].T)-train_label[j]*(alpha[j]-alpha_j_old)*train[i].dot(train[j].T)
                b2=b-E_j-train_label[j]*(alpha[j]-alpha_j_old)*train[j].dot(train[j].T)-train_label[i]*(alpha[i]-alpha_i_old)*train[j].dot(train[i].T)
                if alpha[i]>0 and alpha[j]<C:
                    b=b1
                if alpha[j]>0 and alpha[j]<C:
                    b=b2
                else:
                    b=(b2+b1)/2
                change_two+=1
        if change_two==0:
            Iter+=1
        else:
            Iter=0
    return w,b
            

In [7]:
import pandas as pd
import numpy as np
file=pd.read_csv('C:\\Users\\zhou\\Desktop\\train.csv')
file1=pd.read_csv('C:\\Users\\zhou\\Desktop\\test.csv')
file2=pd.read_csv('C:\\Users\\zhou\\Desktop\\gender_submission.csv')

In [8]:
data_train=file.loc[0:,['Pclass','Sex','Age','SibSp','Parch','Fare']]
data_train_label=file.loc[0:,'Survived']
data_test=file1.loc[0:,['Pclass','Sex','Age','SibSp','Parch','Fare']]
data_test_label=file2.iloc[0:,0:]

In [9]:
count=0
for i in data_train.loc[0:,'Sex']:
    count+=1
    if i=='female':
        data_train.loc[0:,'Sex'][count-1]=0
    if i=='male':
        data_train.loc[0:,'Sex'][count-1]=1
count=0
for i in data_test.loc[0:,'Sex']:
    count+=1
    if i=='female':
        data_test.loc[0:,'Sex'][count-1]=0
    if i=='male':
        data_test.loc[0:,'Sex'][count-1]=1

In [10]:
data_others=np.array(data_train)[0:,[0,1,3,4,5]]
data_age=np.array(data_train)[0:,[2]]
data_test_others=np.array(data_test)[0:,[0,1,3,4,5]]
data_test_age=np.array(data_test)[0:,[2]]

In [11]:
nan_index_list=[]
for i in range(891):
    if pd.isnull(data_age[i][0]):
        nan_index_list.append(i)  
nan_test_index_list=[]
for i in range(418):
    if pd.isnull(data_test_age[i][0]):
        nan_test_index_list.append(i) 

In [12]:
#归一化处理
data_others=(data_others-data_others.min(axis=0))/(data_others.max(axis=0)-data_others.min(axis=0))
data_test_others=(data_test_others-data_test_others.min(axis=0))/(data_test_others.max(axis=0)-data_test_others.min(axis=0))

C:\Users\zhou\anaconda3\lib\site-packages\numpy\core\_methods.py:34: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial, where)
C:\Users\zhou\anaconda3\lib\site-packages\numpy\core\_methods.py:30: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial, where)


In [13]:
#利用knn算法补充age的缺失值
def knn(nums,data_others,k,data_age):
    data_size=data_others.shape[0]
    diffmat=np.tile(data_others[nums],(data_size,1))-data_others
    diffmat_sqr=diffmat**2
    sqrdiff=diffmat_sqr.sum(axis=1)
    distance_set=sqrdiff**0.5
    distance_sort=np.argsort(distance_set)
    list_near_average=[]
    for i in range(k):
        list_near_average.append(distance_sort[i])
    average=[]
    for i in list_near_average:
        average.append(data_age[i][0])
    average_new=[]
    for i in average:
        if pd.notnull(i):
            average_new.append(i)
    return sum(average_new)/len(average_new)
for nums in nan_index_list:
    data_train.loc[0:,'Age'][nums]=knn(nums,data_others,10,data_age)
for nums1 in nan_test_index_list:
    data_test.loc[0:,'Age'][nums1]=knn(nums1,data_test_others,10,data_test_age)

In [14]:
data_train=np.array(data_train)
data_train_label=np.array(data_train_label)

In [15]:
data_test=np.array(data_test)
data_test_label=np.array(data_test_label)
data_test_label=data_test_label[0:,1]

In [22]:
data_train=(data_train-data_train.min(axis=0))/(data_train.max(axis=0)-data_train.min(axis=0))

In [23]:
data_test=(data_test-data_test.min(axis=0))/(data_test.max(axis=0)-data_test.min(axis=0))

In [18]:
for i in range(len(data_train_label)):
    if data_train_label[i]==0:
        data_train_label[i]=-1
        

In [19]:
for i in range(len(data_test_label)):
    if data_test_label[i]==0:
        data_test_label[i]=-1

In [24]:
w,b=smo(data_train,data_train_label,0.6,0.1,50)

In [25]:
count=0
count1=0
for sample in data_test:
    count+=1
    fx=sample.dot(w)+b
    if fx[0]>0:
        a=1
    else:
        a=-1
    if a==data_test_label[count-1]:
        count1+=1
print(count1/count)
        
    

0.8971291866028708
